In [1]:
#TESTING LLAMA 3 

import ollama

# Initialize the model
model = "llama3:8b-instruct-q4_K_M"

# Create a list to store the conversation history
conversation = []

# First interaction
conversation.append({"role": "user", "content": "Hello"})
response = ollama.chat(model=model, messages=conversation)
print(response['message']['content'])

# Add the model's response to the conversation history
conversation.append(response['message'])

# Subsequent interaction
conversation.append({"role": "user", "content": "Tell me about elephants"})
response = ollama.chat(model=model, messages=conversation)
print(response['message']['content'])

# Again, add the model's response to the conversation history
conversation.append(response['message'])

Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?
Elephants are indeed magnificent creatures!

Here are some fascinating facts about elephants:

1. **Largest Land Animal**: Elephants are the largest land animals on Earth, with African elephants weighing up to 6 tons (12,000 lbs) and Asian elephants weighing up to 5 tons (10,000 lbs).
2. **Longest Lifespan**: Elephants have one of the longest lifespans in the animal kingdom, with a average lifespan of 50-70 years in the wild and up to 80 years in captivity.
3. **Intelligent Brains**: Elephants have massive brains, with some weighing around 5 kg (11 lbs), which is larger than that of humans. They are known for their exceptional memory, problem-solving skills, and social intelligence.
4. **Social Structure**: Elephants live in complex societies with a strict hierarchy, led by matriarchal herds. These herds are typically made up of related females, their offspring, and sometimes male relatives.

In [16]:
pip install langchain_community


Note: you may need to restart the kernel to use updated packages.


In [3]:
import chromadb

# Initialize Chroma DB client
client = chromadb.PersistentClient(path='database')

In [5]:
my_collection = client.create_collection(
        name="collection1",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

In [4]:

client.delete_collection(name="collection1")

CHUNKER BASE CLASS

In [6]:
import re
from langchain_community.embeddings import HuggingFaceEmbeddings
class ChunkerBase:
    def __init__(self, model_name="all-MiniLM-L6-v2", max_chunk_size=250):
        self.embeddings_model = HuggingFaceEmbeddings(model_name=model_name)
        self.chunks = []
        self.ids=[]
        self.chunk_embeddings = []
        self.collection = my_collection  # Use ChromaDB collection for storing documents
        self.max_chunk_size = max_chunk_size

    def embed_chunks(self):
        print("Num of input chunks: ", len(self.chunks))
        self.chunk_embeddings = self.embeddings_model.embed_documents(self.chunks)
        print("Num of retrieved embeddings for chunks: ", len(self.chunk_embeddings))
        documents=[]
        # list(map(str, self.chunks))
        # embeddings_as_lists = [embedding for embedding in self.chunk_embeddings]


        self.ids = [f"idx{i+1}" for i in range(len(self.chunks))]

        print(self.chunks)
        print(self.ids)
        # self.collection.add(self.chunks,self.ids)
        self.collection.add(embeddings=self.chunk_embeddings, documents=self.chunks, ids=self.ids)

    

    def embed_query(self, query):
        return self.embeddings_model.embed_documents([query])[0]

    def search_similar_chunks(self, query_embedding, top_k=7):
        print(query_embedding)
        results = self.collection.query(query_embedding, n_results=top_k)

        if not results:
            print("No similar chunks found.")
            return [], []

        print(results)

        similar_chunks = []
        for doc_list, score_list in zip(results['documents'], results['distances']):
            for doc, score in zip(doc_list, score_list):
                similar_chunks.append((doc, score))

        return similar_chunks


    def get_all_chunks(self):
        return self.chunks

RECURSIVE CHUNKER CLASS 

In [7]:
class RecursiveChunker(ChunkerBase):
    def __init__(self, model_name="all-MiniLM-L6-v2", max_chunk_size=150, max_paragraph_words=100):
        super().__init__(model_name, max_chunk_size)
        self.max_paragraph_words = max_paragraph_words

    def _split_sentences(self, text):
        return re.split(r'(?<=[.?!])\s+', text)

    def recursive_chunk_text(self, text):
        words = text.split()

        if len(words) <= self.max_chunk_size:
            return [text]

        # Find the best splitting point
        split_point = self.max_chunk_size
        while split_point > 0 and not words[split_point - 1].endswith(('.', '!', '?')):
            split_point -= 1

        if split_point == 0:  # No good splitting point found, force split at max_chunk_size
            split_point = self.max_chunk_size

        # Split the text
        chunk = ' '.join(words[:split_point])
        remaining_text = ' '.join(words[split_point:])

        # Recursive call
        chunks = [chunk] + self.recursive_chunk_text(remaining_text)

        return chunks

    def chunk_text(self, text):
        paragraphs = text.split('\n\n')
        self.chunks = []

        for paragraph in paragraphs:
            if len(paragraph.split()) > self.max_paragraph_words:
                indented_chunks = self.recursive_chunk_text(paragraph)
                self.chunks.extend(indented_chunks)
            else:
                self.chunks.append(paragraph)

        print('Total number of chunks:', len(self.chunks))

    def read_text_from_file(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        return text

# Initialize the chunker
chunker = RecursiveChunker()

# File path to read
file_path = 'inputFile.txt'

# Read text from file
text_from_file = chunker.read_text_from_file(file_path)

# Chunk the text
chunker.chunk_text(text_from_file)
chunker.embed_chunks()

print('Total number of chunks:', len(chunker.chunks))




c:\Users\MY IT STORE\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\MY IT STORE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\MY IT STORE\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be re

Total number of chunks: 65
Num of input chunks:  65
Num of retrieved embeddings for chunks:  65
['Introduction to Quantum Physics', 'Quantum physics, also known as quantum mechanics or quantum theory, is a fundamental branch of physics that deals with the behavior of matter and energy at the smallest scales, typically at the level of atoms and subatomic particles. Developed in the early 20th century, quantum physics has revolutionized our understanding of the universe, providing insights into the nature of reality that are often counterintuitive and challenging to grasp. Quantum physics, also known as quantum mechanics or quantum theory, is a fundamental branch of physics that deals with the behavior of matter and energy at the smallest scales, typically at the level of atoms and subatomic particles. Developed in the early 20th century, quantum physics has revolutionized our understanding of the universe, providing insights into the nature of reality that are often counterintuitive and

PRINTING SIMILAR CHUNKS TO QUERY

In [8]:
# Query similar chunks
query = "What are applications of Quantum Physics?"
query_embedding = chunker.embed_query(query)

# Search for similar chunks
similar_chunks = chunker.search_similar_chunks(query_embedding,10 )

print(f"Top {len(similar_chunks)} Similar Chunks for Query: '{query}'")
print("=" * 50)
for i, (chunk_content, similarity_score) in enumerate(similar_chunks, start=1):
    print(f"{i}. Similarity Score: {similarity_score:.4f}")
    print(f"   Chunk Content: {chunk_content}")
    print("-" * 50)

[-0.06603599339723587, 0.008933143690228462, -0.01955753192305565, 0.00827786698937416, -0.08145613223314285, 0.028088178485631943, -0.03494895622134209, 0.0029880423098802567, 0.0003813735966105014, 0.03194334730505943, -0.004551070276647806, -0.05173907428979874, -0.12207017093896866, 0.045684974640607834, 0.02690632827579975, 0.02667168900370598, 0.053102537989616394, 0.04475398734211922, -0.013978607021272182, -0.08003983646631241, -0.04613404721021652, -0.010109159164130688, 0.03960271179676056, -0.005283980630338192, -0.02564314194023609, -0.05054779723286629, 0.004721928853541613, -0.017389889806509018, 0.026274198666214943, 0.010404972359538078, 0.028804486617445946, 0.08532766252756119, 0.022079095244407654, -0.010746047832071781, -0.04287583380937576, 0.016960224136710167, 0.029728632420301437, -0.01653442159295082, 0.018643852323293686, -0.06052058935165405, -0.03087843582034111, -0.03969278931617737, -0.02085341326892376, 0.005707853473722935, 0.04509488493204117, -0.057296

USING CUSTOM TEMPLATE TO SEND PROMPT TO LLM

In [9]:
result_string = " ".join([text for text, score in similar_chunks])

# print(result_string)
import ollama

# Initialize the model
model = "llama3:8b-instruct-q4_K_M"

# Create a list to store the conversation history
conversation = []
query="Tell me about Schrodinger Equation?"
setrule="Give the answer to question below using Text given below after the question . If you can not find the answer in given text say 'I am sorry. i dont know the answer'"
# First interaction
# conversation.append({"role": "user", "content": "Hello"})
convo=setrule+'\n'+query+'\n'+result_string

print(convo)


Give the answer to question below using Text given below after the question . If you can not find the answer in given text say 'I am sorry. i dont know the answer'
Tell me about Schrodinger Equation?
Applications of Quantum Physics Fundamental Concepts of Quantum Physics Quantum Mechanics in Particle Physics: Quantum mechanics is essential for understanding the behavior of elementary particles and their interactions, which are studied in particle physics experiments, such as those conducted at the Large Hadron Collider. Quantum Mechanics in Particle Physics: Quantum mechanics is essential for understanding the behavior of elementary particles and their interactions, which are studied in particle physics experiments, such as those conducted at the Large Hadron Collider. Quantum physics, also known as quantum mechanics or quantum theory, is a fundamental branch of physics that deals with the behavior of matter and energy at the smallest scales, typically at the level of atoms and subatom

SENDING IRRELVENT QUESTION TO THE LLM ACCORDING TO CONTEXT AND CHECKING ITS RESPONSE

In [10]:
print(convo)
conversation.append({"role": "user", "content": convo})
response = ollama.chat(model=model, messages=conversation)

print(response['message']['content'])

Give the answer to question below using Text given below after the question . If you can not find the answer in given text say 'I am sorry. i dont know the answer'
Tell me about Schrodinger Equation?
Applications of Quantum Physics Fundamental Concepts of Quantum Physics Quantum Mechanics in Particle Physics: Quantum mechanics is essential for understanding the behavior of elementary particles and their interactions, which are studied in particle physics experiments, such as those conducted at the Large Hadron Collider. Quantum Mechanics in Particle Physics: Quantum mechanics is essential for understanding the behavior of elementary particles and their interactions, which are studied in particle physics experiments, such as those conducted at the Large Hadron Collider. Quantum physics, also known as quantum mechanics or quantum theory, is a fundamental branch of physics that deals with the behavior of matter and energy at the smallest scales, typically at the level of atoms and subatom

USING CUSTOM TEMPLATE TO SEND PROMPT TO LLM

In [11]:


# Initialize the model
model = "llama3:8b-instruct-q4_K_M"

# Create a list to store the conversation history
conversation = []
query="What are the applications of qunatum physics?"
setrule="Give the answer to question using Text given below . If you can not find the answer in given text say 'I am sorry. i dont know the answer'"
# First interaction
# conversation.append({"role": "user", "content": "Hello"})
convo=setrule+'\n'+query+'\n'+result_string

conversation.append({"role": "user", "content": convo})
response1 = ollama.chat(model=model, messages=conversation)

print(response1['message']['content'])

According to the text, the applications of quantum physics are:

1. Particle Physics: understanding the behavior of elementary particles and their interactions.
2. Astrophysics: explaining various astrophysical phenomena such as the behavior of particles in neutron stars, black holes, and the early universe.
3. Cosmology: explaining fundamental processes that occurred during the formation of the universe.
4. Quantum Computing: performing computations that are infeasible for classical computers.
5. Quantum Sensors: achieving extremely high sensitivity and precision using quantum phenomena such as superposition and entanglement.
6. Quantum Biology: exploring biological processes where quantum effects may play a crucial role, such as photosynthesis, enzyme activity, and avian navigation.

Note that these applications are based on the text provided, and there might be other applications of quantum physics not mentioned in this specific text.


USING SYSTEM USER SYNTAX TO SEND PROMPT TO LLM

In [18]:
import ollama
query1="What are the applications of quantum physics?"
response2 = ollama.chat(model='llama3', messages=[
    {
        'role': 'system',
        'content': f"You are an expert in Quantum Physics. Give the answer to the question using the text given below. If you cannot find the answer in the given text, say 'I am sorry. I don't know the answer.'\n{result_string}"
    },
    {
        'role': 'user',
        'content': query1
    }
])

print(response2['message']['content'])

According to the given text, some of the applications of Quantum Physics include:

1. **Quantum Mechanics in Particle Physics**: Essential for understanding the behavior of elementary particles and their interactions, which are studied in particle physics experiments, such as those conducted at the Large Hadron Collider.
2. **Astrophysics**: Explains various astrophysical phenomena, such as the behavior of particles in neutron stars, black holes, and the early universe.
3. **Cosmology**: Helps explain the fundamental processes that occurred during the formation of the universe, such as cosmic inflation and the behavior of particles in the early universe.
4. **Quantum Computing**: Leverages superposition and entanglement to perform computations that are infeasible for classical computers, with potential applications in cryptography, optimization, and material science.
5. **Quantum Sensors**: Utilizes quantum phenomena like superposition and entanglement to achieve extremely high sensiti

Using PROMPT SYNTAX TO CREATE A PROMPT AND SEND TO LLM

In [25]:


# Initialize the model
model = "llama3:8b-instruct-q4_K_M"

# Create a list to store the conversation history
conversation = []

user_query = "What are the applications of quantum physics?"

system_part = f"You are an expert in Quantum Physics. Give the answer to the question using the text given below. If you cannot find the answer in the given text, say 'I am sorry. I don't know the answer.'\n{result_string}"

prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_part}\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_query}\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>"   


conversation.append({"role": "assistant", "content": prompt})
response3 = ollama.chat(model=model, messages=conversation)

print(response3['message']['content'])








According to the provided text, some of the applications of quantum physics include:

1. Quantum Mechanics in Particle Physics: Understanding the behavior of elementary particles and their interactions.
2. Quantum Mechanics in Astrophysics: Explaining various astrophysical phenomena such as neutron stars, black holes, and the early universe.
3. Quantum Mechanics in Cosmology: Explaining the fundamental processes that occurred during the formation of the universe, such as cosmic inflation and the behavior of particles in the early universe.
4. Quantum Computing: Leveraging superposition and entanglement to perform computations infeasible for classical computers, revolutionizing fields like cryptography, optimization, and material science.
5. Quantum Sensors: Utilizing quantum phenomena to achieve high sensitivity and precision, used in applications such as gravimeters, accelerometers, and magnetometers in geophysics, medicine, and other fields.
6. Quantum Biology: Exploring the role o

GIVING LARGE CONTEXT AND FINDING ANSWER FROM IT USING LLM

In [15]:
context = """
The 2023 FIFA Women's World Cup was the ninth edition of the FIFA Women's World Cup, the quadrennial international women's
football championship contested by women's national teams and organised by FIFA. The tournament, which took place from
20 July to 20 August 2023, was jointly hosted by Australia and New Zealand.[2][3][4] It was the first FIFA Women's World Cup
with more than one host nation, as well as the first World Cup to be held across multiple confederations, as Australia is in the
Asian confederation, while New Zealand is in the Oceanian confederation. It was also the first Women's World Cup to be held
in the Southern Hemisphere.[5]
This tournament was the first to feature an expanded format of 32 teams from the previous 24, replicating the format used
for the men's World Cup from 1998 to 2022.[2] The opening match was won by co-host New Zealand, beating Norway at Eden Park
in Auckland on 20 July 2023 and achieving their first Women's World Cup victory.[6]
Spain were crowned champions after defeating reigning European champions England 1–0 in the final. It was the first time a
European nation had won the Women's World Cup since 2007 and Spain's first title, although their victory was marred by the
Rubiales affair. Spain became the second nation to win both the women's and men's World Cup since Germany in the 2003 edition.
[10] In addition, they became the first nation to concurrently hold the FIFA women's U-17, U-20, and senior World Cups.[11]
Sweden would claim their fourth bronze medal at the Women's World Cup while co-host Australia achieved their best placing yet,
finishing fourth.[12] Japanese player Hinata Miyazawa won the Golden Boot scoring five goals throughout the tournament. Spanish
player Aitana Bonmatí was voted the tournament's best player, winning the Golden Ball, whilst Bonmatí's teammate Salma
Paralluelo was awarded the Young Player Award. England goalkeeper Mary Earps won the Golden Glove, awarded to the best-performing
goalkeeper of the tournament.
Of the eight teams making their first appearance, Morocco were the only one to advance to the round of 16 (where they lost
to France; coincidentally, the result of this fixture was similar to the men's World Cup in Qatar, where France defeated
Morocco in the semi-final). The United States were the two-time defending champions,[13] but were eliminated in the round of
16 by Sweden, the first time the team had not made the semi-finals at the tournament, and the first time the defending
champions failed to progress to the quarter-finals.[14]
Australia's team, nicknamed the Matildas, performed better than expected, and the event saw many Australians unite to
support them. The Matildas, who beat France to make the semi-finals for the first time, saw record numbers of fans watching
their games, their 3–1 loss to England becoming the most watched television broadcast in Australian history, with an average
viewership of 7.13 million and a peak viewership of 11.15 million viewers.[18]
It was the most attended edition of the competition ever held.
"""

prompt = f"""
Given the following context, who won the 2023 Women's World cup?
context: {context}
"""


conversation = []
query="Given the following context, who won the 2023 Women's World cup?"
setrule="Give the answer to question using Text given below . If you can not find the answer in given text say 'I am sorry. i dont know the answer'"
# First interaction
# conversation.append({"role": "user", "content": "Hello"})
convo=query+'\n'+context

conversation.append({"role": "user", "content": convo})
response1 = ollama.chat(model=model, messages=conversation)

print(response1['message']['content'])

According to the text, Spain won the 2023 Women's World Cup by defeating England 1-0 in the final.


USING LANGCHAIN PROMPT TEMPLATE TO SEND PROMPT TO LLM

In [14]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Assume you have these from your RAG implementation
relevant_chunks = ["chunk1 content", "chunk2 content", "chunk3 content"]
user_query = "What are the applications of quantum physics??"

# Create the prompt template
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}
Answer:"""

prompt = ChatPromptTemplate.from_template(prompt_template)

# Initialize the LLM
llm = ChatOllama(model="llama3:8b-instruct-q4_K_M")

# Create the chain
chain = prompt | llm | StrOutputParser()

# Prepare the context
# context = "\n\n".join(similar_chunks)

# Invoke the chain
response = chain.invoke({
    "context": result_string,
    "question": user_query
})

print(response)

According to the provided context, some of the applications of quantum physics include:

* Quantum Mechanics in Particle Physics: understanding the behavior of elementary particles and their interactions
* Quantum Mechanics in Astrophysics: explaining various astrophysical phenomena such as the behavior of particles in neutron stars, black holes, and the early universe
* Quantum Mechanics in Cosmology: explaining fundamental processes that occurred during the formation of the universe
* Quantum Computing: leveraging superposition and entanglement to perform computations that are infeasible for classical computers
* Quantum Sensors: utilizing quantum phenomena like superposition and entanglement to achieve extremely high sensitivity and precision, with applications in fields such as geophysics, medical diagnostics, etc.
* Quantum Biology: exploring the role of quantum mechanics in biological processes like photosynthesis, enzyme activity, and avian navigation.

Note that this is not an 